# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output1'
TRAIT = 'Heart-rate'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

utils.py has been loaded


['TCGA_Colon_and_Rectal_Cancer_(COADREAD)']

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Heart-rate"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE117070',
 'GSE122279',
 'GSE12385',
 'GSE18583',
 'GSE236927',
 'GSE34788',
 'GSE35661',
 'GSE72462']

#1 error with axis, #2 too long prompt for chat gpt, #3 ValueError: attempt to get argmin of an empty sequence, #4 MemoryError: Unable to allocate 8.24 MiB for an array with shape (1079861,) and data type int64, #4 memory error, #5 ParserError: Error tokenizing data. C error: out of memory, #6 MemoryError:  

Repeat the below steps for all the accession numbers

In [53]:
cohort = accession_num = "GSE72462"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Heart-rate\\GSE72462\\GSE72462_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Heart-rate\\GSE72462\\GSE72462_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [54]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"TGFÎ² contributes to impaired exercise response by suppression of mitochondrial key regulators in skeletal muscle"
!Series_summary	"substantial number of people at risk to develop type 2 diabetes could not improve insulin sensitivity by physical training intervention. We studied the mechanisms of this impaired exercise response in 20 middle-aged individuals who performed a controlled eight weeks cycling and walking training at 80 % individual VO2max. Participants identified as non-responders in insulin sensitivity (based on Matsuda index) did not differ in pre-intervention parameters compared to high responders. The failure to increase insulin sensitivity after training correlates with impaired up-regulation of mitochondrial fuel oxidation genes in skeletal muscle, and with the suppression of the upstream regulators PGC1Î± and AMPKÎ±2. The muscle transcriptome of the non-responders is further characterized by an activation of TGFÎ² and TGFÎ² target genes, which is associ

In [55]:
clinical_data.head()

,!Sample_geo_accession,GSM1862914,GSM1862915,GSM1862916,GSM1862917,GSM1862918,GSM1862919,GSM1862920,GSM1862921,GSM1862922,...,GSM1862940,GSM1862941,GSM1862942,GSM1862943,GSM1862944,GSM1862945,GSM1862946,GSM1862947,GSM1862948,GSM1862949
0,!Sample_characteristics_ch1,insulin sensitivity: non-responder,insulin sensitivity: high-responder,insulin sensitivity: high-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: low-responder,...,insulin sensitivity: low-responder,insulin sensitivity: low-responder,insulin sensitivity: high-responder,insulin sensitivity: low-responder,insulin sensitivity: high-responder,insulin sensitivity: high-responder,insulin sensitivity: non-responder,insulin sensitivity: high-responder,insulin sensitivity: high-responder,insulin sensitivity: low-responder
1,!Sample_characteristics_ch1,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,...,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle,tissue: muscle
2,!Sample_characteristics_ch1,Sex: female,Sex: female,Sex: male,Sex: male,Sex: female,Sex: female,Sex: female,Sex: male,Sex: female,...,Sex: female,Sex: male,Sex: female,Sex: male,Sex: female,Sex: male,Sex: female,Sex: female,Sex: female,Sex: female
3,!Sample_characteristics_ch1,age: 62,age: 61,age: 37,age: 40,age: 24,age: 48,age: 37,age: 42,age: 43,...,age: 43,age: 39,age: 45,age: 54,age: 40,age: 58,age: 45,age: 56,age: 64,age: 28


In [56]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['insulin sensitivity: non-responder',
  'insulin sensitivity: high-responder',
  'insulin sensitivity: low-responder'],
 1: ['tissue: muscle'],
 2: ['Sex: female', 'Sex: male'],
 3: ['age: 62',
  'age: 61',
  'age: 37',
  'age: 40',
  'age: 24',
  'age: 48',
  'age: 42',
  'age: 43',
  'age: 39',
  'age: 45',
  'age: 54',
  'age: 58',
  'age: 56',
  'age: 64',
  'age: 28']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [57]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Heart-rate\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Heart-rate\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an inte

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [104]:
is_gene_availabe = True
trait_row = None
age_row = 3
gender_row = 2

trait_type = None

In [105]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [85]:
# def convert_trait(tissue_type):
#     """
#     Convert tissue type to epilepsy presence (binary).
#     Assuming epilepsy presence for 'Hippocampus' tissue.
#     """
#     if tissue_type == 'primary diagnosis: PSORIASIS':
#         return 1  # Epilepsy present
#     else:
#         return 0  # Epilepsy not present

# def convert_age(age_string):
#     """
#     Convert age string to a numerical value.
#     """
#     if age_string.startswith('age: '):
#         try:
#             return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
#         except ValueError:
#             return None  # In case of parsing error
#     else:
#         return None  # If age string does not follow expected format

# def convert_gender(gender_string):
#     """
#     Convert gender string to a binary representation.
#     Female -> 0, Male -> 1, Unknown -> None
#     """
#     if gender_string == 'Sex: Female':
#         return 0
#     elif gender_string == 'Sex: Male':
#         return 1
#     else:
#         return None  # In case of unknown gender

# # Example usage
# example_tissue = 'tissue: Hippocampus'
# example_age = 'age: 23y'
# example_gender = 'gender: Male'

# converted_trait = convert_trait(example_tissue)
# converted_age = convert_age(example_age)
# converted_gender = convert_gender(example_gender)

# converted_trait, converted_age, converted_gender

# def convert_age(value):
#     try:
#         return int(value)
#     except ValueError:
#         return None

# def convert_gender(value):
#     if value.lower() == 'male':
#         return 'male'
#     elif value.lower() == 'female':
#         return 'female'
#     else:
#         return None

# def convert_trait(value):
#     try:
#         return float(value)
#     except ValueError:
#         return None

def convert_trait(value):
    # Check if the value is in the list of unique trait values
    trait_values = [
        'insulin sensitivity: non-responder',
        'insulin sensitivity: high-responder',
        'insulin sensitivity: low-responder'
    ]
    if value in trait_values:
        return value  # If the value is valid, return it
    else:
        return None  # Otherwise, return None for unknown values

def convert_age(value):
    try:
        # Convert the value to an integer if possible
        return int(value.split(':')[-1])
    except ValueError:
        return None  # Return None for invalid or unknown values

def convert_gender(value):
    # Check if the value is in the list of unique gender values
    gender_values = ['Sex: female', 'Sex: male']
    if value in gender_values:
        return value.split()[-1]  # Extract the gender from the value
    else:
        return None  # Return None for unknown values

def convert_age(value):
    try:
        return int(value.split(':')[1]) if value else None
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    if value and 'female' in value.lower():
        return 'Female'
    elif value and 'male' in value.lower():
        return 'Male'
    else:
        return None





In [86]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1862914,GSM1862915,GSM1862916,GSM1862917,GSM1862918,GSM1862919,GSM1862920,GSM1862921,GSM1862922,GSM1862923,...,GSM1862940,GSM1862941,GSM1862942,GSM1862943,GSM1862944,GSM1862945,GSM1862946,GSM1862947,GSM1862948,GSM1862949
Heart-rate,insulin sensitivity: non-responder,insulin sensitivity: high-responder,insulin sensitivity: high-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: non-responder,insulin sensitivity: low-responder,insulin sensitivity: low-responder,...,insulin sensitivity: low-responder,insulin sensitivity: low-responder,insulin sensitivity: high-responder,insulin sensitivity: low-responder,insulin sensitivity: high-responder,insulin sensitivity: high-responder,insulin sensitivity: non-responder,insulin sensitivity: high-responder,insulin sensitivity: high-responder,insulin sensitivity: low-responder
Age,62,61,37,40,24,48,37,42,43,39,...,43,39,45,54,40,58,45,56,64,28
Gender,Female,Female,Male,Male,Female,Female,Female,Male,Female,Male,...,Female,Male,Female,Male,Female,Male,Female,Female,Female,Female


### Genetic data preprocessing and final filtering

In [87]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1862914,GSM1862915,GSM1862916,GSM1862917,GSM1862918,GSM1862919,GSM1862920,GSM1862921,GSM1862922,GSM1862923,...,GSM1862940,GSM1862941,GSM1862942,GSM1862943,GSM1862944,GSM1862945,GSM1862946,GSM1862947,GSM1862948,GSM1862949
ID,,,,,,,,,,,,,,,,,,,,,
2824546_st,11.404,11.525,11.122,11.449,11.425,11.454,11.553,11.396,11.396,11.281,...,11.255,11.537,11.445,11.454,11.401,11.419,11.147,11.306,11.241,11.195
2824549_st,11.627,11.791,11.279,11.465,11.230,11.560,11.445,11.188,11.329,11.286,...,11.418,11.180,11.417,11.584,11.775,11.103,11.279,11.638,11.485,11.266
2824551_st,10.918,11.109,11.116,11.186,11.091,11.217,11.285,11.199,11.204,11.184,...,10.950,11.234,10.990,11.196,11.184,11.228,10.996,10.926,10.945,11.201
2824554_st,12.011,12.030,11.834,11.815,11.849,11.907,11.792,11.693,11.906,12.044,...,11.800,11.763,11.823,11.867,12.077,11.714,11.904,12.079,11.987,11.820
2827992_st,11.723,11.728,11.745,11.737,11.887,11.707,11.858,11.350,11.803,11.685,...,11.662,12.061,11.688,11.796,11.891,11.736,11.611,11.683,11.511,11.780


In [88]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['2824546_st',
 '2824549_st',
 '2824551_st',
 '2824554_st',
 '2827992_st',
 '2827995_st',
 '2827996_st',
 '2828010_st',
 '2828012_st',
 '2835442_st',
 '2835447_st',
 '2835453_st',
 '2835456_st',
 '2835459_st',
 '2835461_st',
 '2839509_st',
 '2839511_st',
 '2839513_st',
 '2839515_st',
 '2839517_st']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [89]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st', '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st', '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st', '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st']\n"

If not required, jump directly to the gene normalization step

In [90]:
requires_gene_mapping = True

In [91]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1302-9 // microRN

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [92]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287

In [93]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'gene_assignment'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [94]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [100]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [101]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 36 samples.


In [103]:
merged_data

,Heart-rate,Age,Gender,BET1L,BOLA2,BOLA2B,CA14,CD79A,CLK3,EOS,...,SPRR2C,SPRR2D,SPRR2F,TBP,TPR,U1,GLA,HASPIN,SP1,TESMIN
GSM1862914,insulin sensitivity: non-responder,62,Female,7.003,6.772,6.772,2.463,5.617,3.367,4.194,...,5.834,5.834,5.834,4.291113,4.693375,6.099,5.108,2.298,4.367226,2.325
GSM1862915,insulin sensitivity: high-responder,61,Female,6.675,6.965,6.965,2.497,5.632,3.325,4.173,...,5.916,5.916,5.916,4.344894,4.563375,6.099,5.128,2.508,4.240194,2.343
GSM1862916,insulin sensitivity: high-responder,37,Male,6.761,6.941,6.941,2.324,5.604333,3.432,4.233,...,5.649,5.649,5.649,4.336549,4.572125,5.889,5.264,2.45,4.283645,2.312
GSM1862917,insulin sensitivity: non-responder,40,Male,7.428,6.868,6.868,2.47,5.896,3.317,4.093,...,5.862,5.862,5.862,4.377606,4.536125,6.039,5.288,2.71,4.318081,2.269
GSM1862918,insulin sensitivity: non-responder,24,Female,6.925,6.893,6.893,2.413,5.765333,3.41,4.048,...,5.862,5.862,5.862,4.320232,4.631375,6.13,5.24,2.622,4.300435,2.359
GSM1862919,insulin sensitivity: non-responder,48,Female,7.009,6.96,6.96,2.379,5.680333,3.203,4.044,...,5.862,5.862,5.862,4.332951,4.6465,6.154,5.195,2.462,4.367403,2.31
GSM1862920,insulin sensitivity: non-responder,37,Female,7.343,7.1,7.1,2.642,5.872667,3.406,4.074,...,5.915,5.915,5.915,4.38881,4.456875,5.918,5.314,2.577,4.324952,2.305
GSM1862921,insulin sensitivity: non-responder,42,Male,6.875,6.682,6.682,2.469,5.588,3.242,4.064,...,5.66,5.66,5.66,4.367056,4.6045,6.057,5.337,2.627,4.344887,2.289
GSM1862922,insulin sensitivity: low-responder,43,Female,7.149,6.787,6.787,2.556,5.710667,3.089,4.022,...,5.815,5.815,5.815,4.362817,4.642875,6.11,5.24,2.564,4.324581,2.375
GSM1862923,insulin sensitivity: low-responder,39,Male,7.08,7.246,7.246,2.419,5.617667,3.174,4.07,...,5.637,5.637,5.637,4.377035,4.40125,5.952,5.24,2.716,4.323468,2.379


In [98]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [51]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [52]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
from utils import *

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE123088'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE123088,True,True,False,True,True,192,
1,GSE123086,True,True,False,True,True,154,


In [ ]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Psoriasis,Age,Gender,-0.001001579,-0.010425045,-0.045320336,-0.046967808,-0.069891929,-0.081080626,-0.093186199,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,0.0,56.0,1.0,3.293947,3.344254,3.049427,3.489313,3.713489,3.182711,2.885457,...,11.631593,9.914236,5.934949,4.756848,10.265351,10.109506,4.033606,11.065690,8.733339,9.681242
1,0.0,20.0,0.0,2.295976,3.490590,3.402046,3.960615,2.295976,3.209781,3.380016,...,11.685883,9.888024,5.981049,3.573693,10.808423,10.135307,3.854406,10.578108,8.207179,9.830063
2,0.0,51.0,0.0,2.878075,3.642838,3.281445,3.630912,2.878075,3.658581,3.466199,...,11.823756,9.977897,5.965869,4.439257,10.669074,10.243314,4.174246,10.824677,8.554421,10.367084
3,0.0,37.0,1.0,3.567006,3.393262,3.477423,3.000214,3.190889,3.070480,2.543753,...,11.871873,9.948733,6.094711,4.479720,10.832092,10.253960,3.735004,10.505996,8.786537,9.695719
4,0.0,61.0,1.0,4.034010,3.607132,2.822304,3.712049,3.126172,3.017512,2.949723,...,11.931741,10.017533,6.085654,3.807143,10.252109,10.396369,4.112947,10.797622,8.826644,8.821104


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

The cross-validation accuracy is 56.84% ± 4.59%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)